In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import joblib

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# ===============================
# LOAD ALIGNED NB15 DATA
# ===============================
train = pd.read_csv("/kaggle/input/cicds-unsw-alligned-dataset/UNSW_aligned_train.csv")
test = pd.read_csv("/kaggle/input/cicds-unsw-alligned-dataset/UNSW_aligned_test.csv")

print("Train:", train.shape, " | Test:", test.shape)

# ===============================
# Ensure attack_cat exists
# ===============================
train["attack_cat"] = pd.to_numeric(train["attack_cat"], errors="coerce")
test["attack_cat"] = pd.to_numeric(test["attack_cat"], errors="coerce")
train.dropna(subset=["attack_cat"], inplace=True)
test.dropna(subset=["attack_cat"], inplace=True)
train["attack_cat"] = train["attack_cat"].astype(int)
test["attack_cat"] = test["attack_cat"].astype(int)

print("Label distribution (train):", Counter(train["attack_cat"]))

# ===============================
# OPTIONAL: BALANCE Dataset
# ===============================
# Drop to max 80k per class (safe for 16GB RAM)
max_samples = 80000
balanced_train = train.groupby("attack_cat", group_keys=False).apply(lambda x: x.sample(min(len(x), max_samples), random_state=42))
balanced_test  = test.groupby("attack_cat", group_keys=False).apply(lambda x: x.sample(min(len(x), max_samples), random_state=42))

print("Balanced train:", balanced_train.shape, " | Balanced test:", balanced_test.shape)

X_train = balanced_train.drop(columns=["attack_cat"])
y_train = balanced_train["attack_cat"]

X_test = balanced_test.drop(columns=["attack_cat"])
y_test = balanced_test["attack_cat"]

# ===============================
# SCALING
# ===============================
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, "nb15_aligned_scaler.pkl")

# ===============================
# CLASS WEIGHTS
# ===============================
classes = np.unique(y_train)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = {classes[i]: float(cw[i]) for i in range(len(cw))}
print("Class Weights:", class_weights)

# ===============================
# MODELS
# ===============================
model_lgb = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.7,
    colsample_bytree=0.7,
    class_weight=class_weights,
    random_state=42,
    n_jobs=-1
)

model_xgb = XGBClassifier(
    objective="multi:softprob",
    num_class=len(classes),
    tree_method="hist",
    n_estimators=300,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.7,
    colsample_bytree=0.7,
    eval_metric="mlogloss",
    random_state=42
)

model_cat = CatBoostClassifier(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    class_weights=list(cw),
    verbose=False
)

model_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=14,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

# ===============================
# TRAIN
# ===============================
print("\nTraining models...")
model_lgb.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)
model_cat.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

joblib.dump(model_lgb, "nb15_lgb.pkl")
joblib.dump(model_xgb, "nb15_xgb.pkl")
joblib.dump(model_cat, "nb15_cat.pkl")
joblib.dump(model_rf, "nb15_rf.pkl")

print("✅ Models saved!")

# ===============================
# ENSEMBLE TESTING
# ===============================
p1 = model_lgb.predict_proba(X_test)
p2 = model_xgb.predict_proba(X_test)
p3 = model_cat.predict_proba(X_test)
p4 = model_rf.predict_proba(X_test)

p_ens = (p1 + p2 + p3 + p4) / 4.0
y_pred = np.argmax(p_ens, axis=1)

print("\n✅ NB15 Ensemble Accuracy:", accuracy_score(y_test, y_pred))
print("✅ NB15 Macro F1:", f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred, zero_division=0))

Train: (82332, 133)  | Test: (175341, 133)
Label distribution (train): Counter({6: 37000, 5: 18871, 3: 11132, 4: 6062, 2: 4089, 7: 3496, 0: 677, 1: 583, 8: 378, 9: 44})


/tmp/ipykernel_37/3673682627.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_train = train.groupby("attack_cat", group_keys=False).apply(lambda x: x.sample(min(len(x), max_samples), random_state=42))
/tmp/ipykernel_37/3673682627.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_test  = test.groupby("attack_cat", group_keys=False).apply(lambda x: x.sample(min(len(x), max_samples), r

Balanced train: (82332, 133)  | Balanced test: (175341, 133)
Class Weights: {0: 12.161299852289513, 1: 14.1221269296741, 2: 2.013499633162142, 3: 0.739597556593604, 4: 1.3581656219069613, 5: 0.4362884849769488, 6: 0.22251891891891892, 7: 2.3550343249427916, 8: 21.78095238095238, 9: 187.11818181818182}

Training models...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010770 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8520
[LightGBM] [Info] Number of data points in the train set: 82332, number of used features: 53
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585


In [2]:
from sklearn.metrics import f1_score

probs = p_ens  # ensemble probabilities
thresholds = []
for c in range(probs.shape[1]):
    cls_mean = np.mean(probs[y_test == c][:, c])
    thresholds.append(cls_mean * 0.8)

thresholds = np.array(thresholds)

adjusted_pred = []
for i in range(len(probs)):
    scaled = probs[i] / thresholds
    adjusted_pred.append(np.argmax(scaled))
adjusted_pred = np.array(adjusted_pred)

print("Tuned Accuracy:", accuracy_score(y_test, adjusted_pred))
print("Tuned Macro F1:", f1_score(y_test, adjusted_pred, average="macro"))
print(classification_report(y_test, adjusted_pred, zero_division=0))

Tuned Accuracy: 0.7528473089579734
Tuned Macro F1: 0.5865543563200181
              precision    recall  f1-score   support

           0       0.05      0.38      0.09      2000
           1       0.11      0.16      0.13      1746
           2       0.35      0.38      0.36     12264
           3       0.86      0.53      0.65     33393
           4       0.64      0.52      0.58     18184
           5       1.00      0.98      0.99     40000
           6       0.87      0.91      0.89     56000
           7       0.90      0.76      0.83     10491
           8       0.59      0.78      0.67      1133
           9       0.66      0.71      0.68       130

    accuracy                           0.75    175341
   macro avg       0.60      0.61      0.59    175341
weighted avg       0.82      0.75      0.78    175341

